## **Chapter 11 - Agentic AI**
This notebook explores Agentic AI through structured examples, showcasing how AI agents plan, retrieve data, and execute tasks autonomously. Using `LangChain` and `CrewAI`, the notebook demonstrates key abstractions, including datasets, prompts, model selection, agent-based reasoning, and task execution. Examples range from structured data retrieval using Hugging Face datasets to interactive AI agents that process information and act independently. By the end, you'll see how these frameworks power AI-driven decision-making and automation, making AI applications more adaptive and autonomous.

**Setting Up API Keys** for Hugging Face, Google, and OpenAI
Before running the code examples in this chapter, API keys must be configured for Hugging Face Hub, Google APIs (Serper and Gemini), and OpenAI. This script retrieves stored credentials in Google Colab Secrets and sets them as environment variables for seamless integration with AI services.

***Note:*** See Google Colab Secrets for instructions on how to store and manage API keys securely.

In [ ]:
# API Key Setup for Hugging Face Hub, Google API, and OpenAI in Google Colab
from google.colab import userdata
import os

# Retrieve Hugging Face API key (using the correct key name)
hf_api_key = userdata.get("HUGGINGFACEHUB_ACCESS_TOKEN")
if hf_api_key:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_api_key  # Standard expected variable name for Hugging Face
    print("✅ Hugging Face API key loaded successfully.")
else:
    raise ValueError("❌ Error: HUGGINGFACEHUB_ACCESS_TOKEN not found in Google Colab secrets. Please set it before running.")

# Retrieve Google Serper API key
serper_api_key = userdata.get("SERPAPI_API_KEY")
if serper_api_key:
    os.environ["SERPAPI_API_KEY"] = serper_api_key
    print("✅ Serper API key loaded successfully.")
else:
    raise ValueError("❌ Error: SERPAPI_API_KEY not found in Google Colab secrets. Please set it before running.")

# Retrieve Google Gemini API key
google_api_key = userdata.get("GEMINI_API_KEY")
if google_api_key:
    os.environ["GEMINI_API_KEY"] = google_api_key
    print("✅ Google Gemini API key loaded successfully.")
else:
    raise ValueError("❌ Error: GEMINI_API_KEY not found in Google Colab secrets. Please set it before running.")


# Retrieve OpenAI API key
openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("✅ OpenAI API key loaded successfully.")
else:
    raise ValueError("❌ Error: OPENAI_API_KEY not found in Google Colab secrets. Please set it before running.")


### Listing 9-1: Loading and Analyzing Game Data
This code retrieves and processes the **Steam Games** dataset from Hugging Face. It selects the top five highest-rated games, sorting them by positive reviews. The dataset provides structured information on game titles, genres, and player feedback, forming the foundation for AI-driven analysis and decision-making.

***Note:*** Before running this code, ensure you have the necessary dependencies installed by running:

In [ ]:
%pip install --quiet datasets

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the Steam Games dataset from Hugging Face
dataset = load_dataset("FronkonGames/steam-games-dataset",
                       split="train")

# Convert to Pandas DataFrame
df = dataset.to_pandas()

# Select relevant columns
df = df[["Name", "Genres", "Positive"]].sort_values(
     by="Positive", ascending=False).head(5)

# Rename columns for clarity
df.columns = ["Game", "Genres", "Positive_Reviews"]

# Display structured dataset output
print("\nTop 5 Highest Rated Games on Steam:")
print(df.to_string(index=False))

### Listing 9-2: Implementing AI-Driven Prompt Templates and Querying Models
This code demonstrates how AI agents interact using structured prompt templates and model queries. Using LangChain's prompt abstractions, it defines different types of interactions—ranging from basic prompts and few-shot learning to dynamic data integration from external sources like Hugging Face datasets. The script then queries LLMs like Mistral and OpenAI models, generating structured responses for trivia, NPC interactions, and game strategy design.

***Note:*** Ensure dependencies are installed before running:

In [ ]:
# Install required packages for Hugging Face and LangChain usage

print("Installing packages... this can take a minute or two.")

%pip install --upgrade --quiet huggingface_hub

%pip install -q langchain langchain-community langchain-huggingface langchain-openai google-search-results

print("All required packaged installed and ready!")

In [ ]:
from huggingface_hub import InferenceClient
from langchain_core.prompts import ChatPromptTemplate
from datasets import load_dataset
import pandas as pd

# Initialize API client and define default model
DEFAULT_MODEL = "mistralai/Mistral-Nemo-Instruct-2407"
client = InferenceClient()

def query_model(messages, model=DEFAULT_MODEL, max_tokens=500,
                temperature=0.7):
    """
    Sends a list of messages to the AI model and returns the response.

    Parameters:
        messages (list): A list of message dictionaries (role, content).
        model (str): The AI model to use (default is Mistral-Nemo).
        max_tokens (int): Maximum number of tokens for the response.
        temperature (float): Controls randomness (higher = more creative).

    Returns:
        str: The AI-generated response.
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    return response.choices[0].message.content

# --- Test Template #1: Wise Old Wizard ---
wizard_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a wise old wizard guiding players on their journey."),
    ('human', "Player Question: {player_question}"),
])

wizard_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a wise old wizard guiding players on their journey."),
    ('human', "Player Question: How do I master fire magic?"),
    ('ai', "To master fire magic, study ancient scrolls in the Ember Library and "
           "practice channeling heat energy through controlled breathing."),
    ('human', "Player Question: What is the best weapon for a sorcerer?"),
    ('ai', "A sorcerer benefits most from a staff infused with elemental power, "
           "such as the Arcane Emberstaff."),
    ('human', "Player Question: {player_question}, answer in 100 words or less.")
])

# Format the wizard prompt example
wizard_message = wizard_prompt.format(
    player_question="How do I master fire magic?"
)

# Test the Wise Old Wizard prompt
wizard_response = query_model([{"role": "user", "content": wizard_message}])
print("\n🔮 Wise Old Wizard Says:\n", wizard_response)

# --- Test Template #2: NPC Dialogue ---
npc_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are an NPC in a fantasy RPG. Guide the player "
               "without giving spoilers."),
    ('human', "Player Info:\n- Class: {player_class}\n- Location: "
              "{current_location}"),
    ('human', "Quest Progress: {quest_progress}"),
    ('human', "Question: {player_question}, answer in 100 words or less.")
])

# Format the NPC dialogue example
npc_message = npc_prompt.format(
    player_class="Mage",
    current_location="Darkwood Forest",
    quest_progress="Searching for the lost artifact",
    player_question="Where should I look next?"
)

# Test NPC prompt template with the model
npc_response = query_model([{"role": "user", "content": npc_message}])
print("\n🧙 NPC Response:\n", npc_response)

# --- Test Template #3: Game Analysis ---
# Load and prepare the dataset
dataset = load_dataset("FronkonGames/steam-games-dataset", split="train")
df = dataset.to_pandas()

# Extract the top-rated games
df = df[["Name", "Genres", "Positive"]].sort_values(
     by="Positive", ascending=False).head(5)

# Format game data into structured input for AI
game_list = "\n".join(f"{row['Name']} (Genre: {row['Genres']}, "
                       f"Reviews: {row['Positive']})"
                       for _, row in df.iterrows())

# Define the prompt template for game analysis
game_prompt = ChatPromptTemplate.from_messages([
    ('system', "Based on the most popular games below, create a simple interactive "
               "game scenario where multiple AI agents play distinct roles."),
    ('human', "Here are the top-rated games:\n{game_list}\n\n"
              "Design a small-scale game scenario suitable for AI agents to play, "
              "including:\n"
              "- Title\n- Setting & Theme\n- Three AI agent roles with unique skills\n"
              "- How these agents interact to complete a shared objective")
])


# Format the game analysis example
game_message = game_prompt.format(game_list=game_list)

# Test game analysis prompt template with the model
game_response = query_model([{"role": "user", "content": game_message}])
print("\n🎮 Game Concept Suggestion:\n", game_response)

### Same as above, instead using OpenAI

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from datasets import load_dataset
import pandas as pd

# Initialize the OpenAI model client
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.7)

def query_model(messages, model=llm, max_tokens=500):
    """
    Sends a list of messages to the AI model and returns the response.

    Parameters:
        messages (list): A list of message dictionaries (role, content).
        model (ChatOpenAI): The OpenAI model instance.
        max_tokens (int): Maximum number of tokens for the response.

    Returns:
        str: The AI-generated response.
    """
    response = model.invoke(messages)
    return response.content

# --- Test Prompt 1: Basic Prompt Structure ---
basic_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a trivia expert who provides clear and informative answers."),
    ('human', "Who invented the lightbulb?"),
    ('ai', "The lightbulb was invented by Thomas Edison in 1879.")
])

# Format the basic prompt example
basic_message = basic_prompt.format()

# Test the Basic Prompt Structure
basic_response = query_model([{"role": "user", "content": basic_message}])
print("\n💡 Trivia Expert Says:\n", basic_response)

# --- Test Prompt 2: Few-Shot Prompting ---
few_shot_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a trivia expert providing clear, well-researched answers."),
    ('human', "Who discovered gravity?"),
    ('ai', "Gravity was discovered by Sir Isaac Newton in the late 17th century."),
    ('human', "What is the speed of light?"),
    ('ai', "The speed of light is approximately 299,792 kilometers per second."),
    ('human', "Who developed the theory of relativity?"),
    ('ai', "Albert Einstein developed the theory of relativity in the early 20th "
           "century."),
    ('human', "Question: {trivia_question}, answer in 100 words or less.")
])

# Format the few-shot prompt example
few_shot_message = few_shot_prompt.format(
    trivia_question="Who discovered penicillin?"
)

# Test the Few-Shot Prompt
few_shot_response = query_model([{"role": "user", "content": few_shot_message}])
print("\n🧪 Few-Shot Trivia Expert Says:\n", few_shot_response)

# --- Test Prompt 3: Prompt with Variables ---
variable_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a trivia expert providing players with challenging questions."),
    ('human', "Category: {trivia_category}"),
    ('human', "Question: {trivia_question}, answer in 100 words or less.")
])

# Format the prompt with variables
variable_message = variable_prompt.format(
    trivia_category="Science",
    trivia_question="What is the speed of light?"
)

# Test the Prompt with Variables
variable_response = query_model([{"role": "user", "content": variable_message}])
print("\n⚡ Trivia Expert Says:\n", variable_response)

# --- Test Prompt 4: Dynamic Data Integration ---
# Load and prepare the dataset
dataset = load_dataset("FronkonGames/steam-games-dataset", split="train")
df = dataset.to_pandas()

# Extract the top-rated games
df = df[["Name", "Genres", "Positive"]].sort_values(
     by="Positive", ascending=False).head(5)

# Format game data into structured input for AI
game_list = "\n".join(f"{row['Name']} (Genre: {row['Genres']}, "
                       f"Reviews: {row['Positive']})"
                       for _, row in df.iterrows())

# Define the prompt template for structured AI gameplay
game_prompt = ChatPromptTemplate.from_messages([
    ('system', "Based on the most popular games below, define a simple game "
               "structure designed for three AI agents."),
    ('human', "Here are the top-rated games:\n{game_list}\n\n"
              "Using this list as inspiration, generate:\n"
              "- A recommended game genre\n"
              "- A title for the game\n"
              "- A simple strategy or turn-based game mechanic\n"
              "- Three AI agent roles: two players and one Game Master (GM)\n"
              "- A brief explanation of how the game is played")
])

# Format the game analysis example
game_message = game_prompt.format(game_list=game_list)

# Test the Prompt with Dynamic Data
game_response = query_model([{"role": "user", "content": game_message}])
print("\n🎮 Game Concept Suggestion:\n", game_response)


### Listing 9-3: Implementing AI Agents and Automating a Trivia Contest
This code sets up Neural Duel, a structured AI-driven trivia game using CrewAI. It defines four agents—a Game Master, two contestants, and a Judge—each assigned specific tasks. The Game Master generates real-time trivia questions, the contestants respond independently, and the Judge evaluates and declares the winner. The flow function ensures data moves correctly between agents, orchestrating a fair and competitive game.

Note: Before running this code, install dependencies below:

In [ ]:
%%capture --no-stderr
%pip install -U --quiet 'crewai[tools]' aisuite

In [ ]:
import re
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from crewai_tools import WebsiteSearchTool

# --- Initialize Web Search Tool ---
search_tool = WebsiteSearchTool()

# --- Define LLMs ---
system_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7,
                        max_tokens=250)
player1_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8,
                         max_tokens=50)
player2_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.9,
                         max_tokens=50)

# --- Define Agents ---

game_master = Agent(
    role="Game Master",
    goal="Generate trivia questions on non-political current events today.",
    backstory="An impartial host creating fair and engaging trivia games.",
    tools=[search_tool],
    llm=system_llm
)

player_1 = Agent(
    role="Contestant 1",
    goal="Answer trivia questions quickly using only internal knowledge.",
    backstory="A fast-thinking trivia player relying on existing knowledge.",
    llm=player1_llm
)

player_2 = Agent(
    role="Contestant 2",
    goal="Answer trivia questions strategically, focusing on reasoning.",
    backstory="A methodical contestant carefully formulating responses.",
    llm=player2_llm
)

judge = Agent(
    role="Judge",
    goal="Evaluate trivia answers and declare a winner with justification.",
    backstory="An impartial adjudicator ensuring fairness in competition.",
    llm=system_llm
)

# --- Define Tasks ---

generate_question_and_answer = Task(
    description="Create a trivia question on a non-political event today. "
                "Ensure the event is recent and unlikely in LLM training data. "
                "Format the response as:\n\n"
                "Question: <trivia question>\n"
                "Answer: <correct answer>",
    expected_output="A clearly formatted trivia question and correct answer.",
    agent=game_master
)

player1_answer = Task(
    description="Given the question:\n{question}\n\n"
                "Provide your best answer using only internal knowledge.",
    expected_output="A concise and accurate response to the trivia question.",
    agent=player_1
)

player2_answer = Task(
    description="Given the question:\n{question}\n\n"
                "Provide your best answer with logical reasoning.",
    expected_output="A thoughtful and well-reasoned response to the trivia "
                    "question.",
    agent=player_2
)

evaluate_and_declare_winner = Task(
    description="Given the question:\n{question}\n\n"
                "Correct answer:\n{right_answer}\n\n"
                "Player 1's answer:\n{player1_answer}\n\n"
                "Player 2's answer:\n{player2_answer}\n\n"
                "Determine the most accurate response and declare the winner.\n\n"
                "Format as:\n\n"
                "Winner: Contestant X\n"
                "Justification: <brief explanation>",
    expected_output="A clear winner declaration and reasoning.",
    agent=judge
)

# --- Define Crews ---

game_master_crew = Crew(
    agents=[game_master],
    tasks=[generate_question_and_answer],
    verbose=True
)

player1_crew = Crew(
    agents=[player_1],
    tasks=[player1_answer],
    verbose=True
)

player2_crew = Crew(
    agents=[player_2],
    tasks=[player2_answer],
    verbose=True
)

judge_crew = Crew(
    agents=[judge],
    tasks=[evaluate_and_declare_winner],
    verbose=True
)

# --- Define Flow Function to Orchestrate Contest ---

def extract_question_answer(response_text):
    """Extracts question and answer from structured text using regex."""
    match = re.search(r"Question:\s*(.*?)\s*Answer:\s*(.*)",
                      response_text, re.IGNORECASE)
    if match:
        return match.group(1).strip(), match.group(2).strip()
    return "No question generated.", "No correct answer provided."

def run_neural_duel():
    """Orchestrates the trivia contest flow between AI agents."""
    print("🎲 Starting Neural Duel!\n")

    # Step 1: Game Master generates trivia question and correct answer
    gm_results = game_master_crew.kickoff()
    print("\n🔍 Raw Game Master Results:\n", gm_results)

    # Extract question and correct answer
    question, correct_answer = extract_question_answer(str(gm_results))

    print(f"\n📝 Trivia Question: {question}")
    print(f"🤫 (Secret) Correct Answer: {correct_answer}")

    # Step 2: Players answer the question
    player1_results = player1_crew.kickoff(inputs={"question": question})
    player2_results = player2_crew.kickoff(inputs={"question": question})

    print(f"\n🏅 Contestant 1's Answer: {str(player1_results)}")
    print(f"🏅 Contestant 2's Answer: {str(player2_results)}")

    # Step 3: Judge evaluates responses and declares the winner
    judge_results = judge_crew.kickoff(inputs={
        "question": question,
        "right_answer": correct_answer,
        "player1_answer": str(player1_results),
        "player2_answer": str(player2_results)
    })

    print("\n🏆 Neural Duel Results:\n", str(judge_results))

# --- Run the Game ---
run_neural_duel()

